## Applied Sequence Modeling with PyTorch (LSTM)
### Predict ICU Stay Length based on lab results using LSTM

Updated 09/26/2024 G. Chism, U of A InfoSci + DataLab

## Install required libraries

For this case we will import _PyTorch_, _sklearn_, _pandas_, and _numpy_.

**To execute code Notebook cells:** Press _SHIFT+ENTER_

In [ ]:
#!pip install -q torch
#!pip install -q scikit-learn
#!pip install -q pandas
#!pip install -q numpy
#!pip install watermark

It's best practice to have all of the libraries loaded at the top of the page

In [30]:
# Import specific classes from PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Import preprocessing from Scikit-Learn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

# Import pandas and numpy
import pandas as pd
import numpy as np

import itertools

Check if we have GPUs available (hint, we probably won't...)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## Data Loading and Preprocessing:

### Converting Data Types:

`pd.to_numeric()` converts columns like los and valuenum to numeric types, coercing any errors (e.g., invalid strings) to NaN.
`pd.to_datetime()` ensures that the charttime and dob columns are properly treated as datetime objects for time-series modeling.

### Handling Missing Values:

After converting the data types, we check for missing values and handle them (in this case, dropping rows with missing values in critical columns like `los` and `valuenum`).


### Check Data Info and Head:

This ensures that the data is now clean and ready for modeling, with no incorrect data types or missing values in critical columns.


In [ ]:
# Assuming the data is already preprocessed via mimic-iii-demo-subset.py and saved as mimic_data.csv
mimic_data = pd.read_csv('data/mimic_data.csv')

# Convert columns to appropriate types
mimic_data['los'] = pd.to_numeric(mimic_data['los'], errors='coerce')  # Convert los to numeric, coercing errors to NaN
mimic_data['valuenum'] = pd.to_numeric(mimic_data['valuenum'], errors='coerce')  # Convert valuenum to numeric
mimic_data['charttime'] = pd.to_datetime(mimic_data['charttime'], errors='coerce')  # Convert charttime to datetime
mimic_data['dob'] = pd.to_datetime(mimic_data['dob'], errors='coerce')  # Convert dob to datetime

# Check for any remaining non-numeric or missing data
print(mimic_data.info())

# Handle missing values (example: drop rows with missing valuenum or los)
clean_data = mimic_data.dropna(subset=['los', 'valuenum'])

print(clean_data.info())

## Group Data by Patients and Time
Sort the data by patient and charttime

In [ ]:
clean_data.sort_values(['subject_id', 'charttime'], inplace=True)

## Create Sequences for LSTM Input

- **Why?** LSTMs require sequential input. We convert each patient’s lab results over time into sequences of defined length (e.g., 10 timesteps) to feed into the LSTM.
- **Normalization**: Scaling the lab values ensures that all features contribute equally during training, improving convergence.

In [ ]:
# Define features (Glucose, Creatinine, Hemoglobin)
features = ['valuenum']

scaler = MinMaxScaler()
clean_data[features] = scaler.fit_transform(clean_data[features])

In [23]:
# Create sequences for LSTM input
def create_sequences(df, sequence_length):
    sequences = []
    targets = []
    for _, group in df.groupby('subject_id'):
        values = group[features].values
        if len(values) >= sequence_length:
            for i in range(len(values) - sequence_length + 1):
                sequences.append(values[i:i+sequence_length])
                targets.append(group['los'].values[0])
    return np.array(sequences), np.array(targets)

# Define sequence length
sequence_length = 10
X, y = create_sequences(clean_data, sequence_length)

print(f'Shape of X (features): {X.shape}')
print(f'Shape of y (target): {y.shape}')

Shape of X (features): (29086, 10, 1)
Shape of y (target): (29086,)


## Prepare DataLoader:

**Why?** PyTorch models require data to be loaded in batches for efficient training. We create a DataLoader to shuffle and batch the sequence data for training.

In [24]:
# Convert numpy arrays to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Create TensorDataset and DataLoader
dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

## Define LSTM Model:

**Why?** We define an LSTM with hidden layers and output a single value (ICU stay length). The model will learn to predict ICU stay based on lab sequences.

**LSTM**: Uses internal memory to capture long-term dependencies in sequential data.

In [25]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h_0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)
        c_0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)
        out, _ = self.lstm(x, (h_0, c_0))
        out = self.fc(out[:, -1, :])
        return out
    
# Hyperparameters
input_size = len(features)
hidden_size = 64
num_layers = 2
output_size = 1

# Initialize model, loss function, and optimizer
model = LSTMModel(input_size, hidden_size, num_layers, output_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Training Loop:

**Why?** The training loop iterates over batches, performing forward and backward passes. We minimize the error between predicted ICU stays and actual stays using MSE loss and backpropagation.

In [26]:
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for batch_x, batch_y in dataloader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        # Forward pass
        outputs = model(batch_x)
        loss = criterion(outputs.squeeze(), batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(dataloader):.4f}')


Epoch 1/20, Loss: 65.0352
Epoch 2/20, Loss: 64.2986
Epoch 3/20, Loss: 64.3002
Epoch 4/20, Loss: 64.2913
Epoch 5/20, Loss: 64.3192
Epoch 6/20, Loss: 64.3026
Epoch 7/20, Loss: 64.2948
Epoch 8/20, Loss: 64.3041
Epoch 9/20, Loss: 64.2893
Epoch 10/20, Loss: 64.3080
Epoch 11/20, Loss: 64.3023
Epoch 12/20, Loss: 64.2875
Epoch 13/20, Loss: 64.2964
Epoch 14/20, Loss: 64.2938
Epoch 15/20, Loss: 64.2900
Epoch 16/20, Loss: 64.2941
Epoch 17/20, Loss: 64.3037
Epoch 18/20, Loss: 64.2765
Epoch 19/20, Loss: 64.2948
Epoch 20/20, Loss: 64.2930


## Evaluation:

**Why?** After training, we evaluate the model’s performance using Mean Absolute Error (MAE), which tells us how close the predicted ICU stay lengths are to the actual lengths.

In [31]:
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for batch_x, batch_y in dataloader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        outputs = model(batch_x)
        predictions.append(outputs.cpu().tolist())
        actuals.append(batch_y.cpu().tolist())

# Flatten the lists before conversion to numpy arrays
predictions = list(itertools.chain.from_iterable(predictions))
actuals = list(itertools.chain.from_iterable(actuals))

# Convert to numpy arrays
predictions = np.array(predictions)
actuals = np.array(actuals)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(actuals, predictions)
print(f'Mean Absolute Error: {mae:.2f}')

Mean Absolute Error: 5.29


## Takeaways

We can interpret that our model was off by 5.29 days, which is not ideal.

### Improvements

1. ***Data Quality and Preprocessing***

- **Handle Missing Values**: Impute missing data or remove incomplete rows.

- **Normalization**: Normalize features to improve convergence.

- **Feature Engineering**: Add relevant features (e.g., demographics, comorbidities). Aggregate lab values over time.

2. ***Increase Training Data***

- Use a larger dataset or augment sequences to provide more data for learning.

3. ***Model Architecture***

- **Increase Complexity**: Add more LSTM layers or neurons.

- **Bi-Directional LSTM**: Learn from both past and future sequences.

- **Attention Mechanism**: Help focus on important parts of the sequence.

- **Dropout Layers**: Prevent overfitting and improve generalization.

4. ***Hyperparameter Tuning***

- Experiment with learning rates, optimizers, batch size, hidden size, and sequence length.

5. ***Regularization***

- **Dropout**: Randomly drop neurons during training to avoid overfitting.

- **L2 Regularization**: Penalize large weights.

6. ***Training Strategies***

- **Early Stopping**: Stop training when validation performance stops improving.

- **Learning Rate Scheduling**: Reduce learning rate during training to improve convergence.

- **Cross-Validation**: Ensure model generalizes well.

7. ***Ensemble Learning***

- Combine multiple models (e.g., LSTMs, GRUs) to improve robustness.

8. ***Domain Knowledge***

- Use domain expertise for feature selection and define custom loss functions to reflect real-world costs.

9. ***Transformer-Based Models***
- Use transformers or hybrid models to capture long-range dependencies better.

10. ***Target Refinement***

- Refine labeling of the target variable (ICU stay length) and consider modeling different types of stays separately.